In [1]:
import sagemaker
import boto3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sagemaker.pytorch import PyTorch
import os

boto_session = boto3.Session(region_name='us-east-1')
sagemaker_session = sagemaker.Session(boto_session=boto_session)

role = "arn:aws:iam::211125439249:role/service-role/AmazonSageMaker-ExecutionRole-20250314T153928"

/opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[03/20/25 03:16:30] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=951938;file:///opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=718432;file:///opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/botocore/credentials.py#1278\1278]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/ubaid/Library/Application Support/sagemaker/config.yaml


[03/20/25 03:16:31] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=721239;file:///opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=64087;file:///opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/botocore/credentials.py#1278\1278]8;;\

In [2]:
df = pd.read_csv('processed_data.csv')

In [3]:
df.head()

,subject_id,prescription_start,presc_embedding,prescription_dose_val_rx,prescription_dose_unit_rx,pre_charttime,pre_ph,pre_pco2,pre_po2,pre_bicarbonate,...,post_fio2_chartevents,post_aado2_calc,post_pao2fio2,post_temperature,post_fio2,post_aado2,post_carboxyhemoglobin,post_methemoglobin,post_calcium,post_intubated
0,10013,2125-10-05T00:00:00,[ 0.89859825 0.5929598 -0.10319789 -0.270541...,0.010317,0,2125-10-04T23:59:00,0.5,0.741935,0.023018,0.0,...,0.866667,0.527567,0.548553,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10013,2125-10-05T00:00:00,[ 1.7285958e-01 2.9352757e-01 -7.4351765e-04 ...,0.011905,1,2125-10-04T23:59:00,0.5,0.741935,0.023018,0.0,...,0.866667,0.527567,0.548553,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10013,2125-10-05T00:00:00,[ 0.95027816 0.3103889 -0.38511172 -0.291505...,0.009921,14,2125-10-04T23:59:00,0.5,0.741935,0.023018,0.0,...,0.866667,0.527567,0.548553,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10013,2125-10-05T00:00:00,[ 0.27488533 0.40669948 0.12023548 -0.101919...,0.017857,0,2125-10-04T23:59:00,0.5,0.741935,0.023018,0.0,...,0.866667,0.527567,0.548553,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10013,2125-10-05T00:00:00,[ 0.20339411 0.47839478 0.05848709 -0.107378...,0.020833,0,2125-10-04T23:59:00,0.5,0.741935,0.023018,0.0,...,0.866667,0.527567,0.548553,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
import pandas as pd
import numpy as np
import re

def clean_and_convert(x):
    if pd.notna(x):
        try:
            # Remove square brackets if they exist
            x = re.sub(r'[\[\]]', '', x)
            # Replace multiple spaces with a single space
            cleaned = re.sub(r'\s+', ' ', x.strip())
            # Convert to numpy array of floats
            return np.array([float(i) for i in cleaned.split(' ')])
        except Exception as e:
            # If there's an error during conversion, print the error and return the original value
            print(f"Error processing value: {x}. Error: {e}")
            return x  # Return the original value in case of error
    return x  # If NaN, return as is

# Check the values before applying the function
print(df['presc_embedding'].head())

# Apply the function to the 'prescription_rx_embeddings' column
df['presc_embedding'] = df['presc_embedding'].apply(clean_and_convert)

# Check the result
df['presc_embedding'][0]

0    [ 0.89859825  0.5929598  -0.10319789 -0.270541...
1    [ 1.7285958e-01  2.9352757e-01 -7.4351765e-04 ...
2    [ 0.95027816  0.3103889  -0.38511172 -0.291505...
3    [ 0.27488533  0.40669948  0.12023548 -0.101919...
4    [ 0.20339411  0.47839478  0.05848709 -0.107378...
Name: presc_embedding, dtype: object


array([ 0.89859825,  0.5929598 , -0.10319789, -0.27054152, -0.3196325 ,
        0.0170421 ,  0.2774623 , -0.3622528 , -0.4726147 , -0.09130935,
       -0.00724437,  0.6437411 ,  0.6121081 , -0.20047094, -0.46266165,
       -0.13812931, -0.2466989 ,  0.11328971, -0.21232367, -0.84981066,
        0.23202187,  1.5324651 ,  1.129424  ,  0.40311688,  0.17756107,
       -0.50952226, -0.334811  ,  0.5386693 ,  0.7032417 ,  0.6361226 ,
        0.9173943 , -1.304354  , -0.99727494, -0.78119403, -0.30380088,
       -0.4214215 ,  0.08080986,  0.08248857, -0.14316714, -0.43057808,
       -0.3460924 , -0.12394918, -0.19472536, -0.79574895, -0.5860874 ,
        0.41286895,  0.46481284,  0.29740804,  0.18613416,  0.52052504,
        1.307526  ,  0.86652833, -0.20474963,  0.58576447,  0.5361008 ,
       -0.6263548 , -0.2791955 ,  0.44079393, -0.62796026, -0.78056127,
        0.2794988 ,  0.22407745, -0.36546314, -0.60245776,  0.5433222 ,
        0.41314498,  0.36379403,  0.30335093,  0.28125036,  0.52

In [5]:
df['presc_embedding'][0].shape

(124,)

In [6]:
# function that gets the unique pairs of subject_id and prescription_start
def get_unique_pairs(df):
    subject_ids = df['subject_id'].unique()
    patient_date_pairs = {id: set() for id in subject_ids}

    for subj in subject_ids:
        df[df['subject_id'] == subj]['prescription_start'].apply(lambda x: patient_date_pairs[subj].add(x))
        # convert set to list
        patient_date_pairs[subj] = list(patient_date_pairs[subj])
    return patient_date_pairs

In [7]:
# function that gets the prescription array with the proper format
def get_presc_input(df):
    prescriptions = []
        
    # Iterate through rows of the DataFrame
    for _, row in df.iterrows():
        # Extract values from each row
        presc = row['presc_embedding']
        dose_val = row['prescription_dose_val_rx']
        dose_unit = row['prescription_dose_unit_rx']
        
        # Concatenate the prescription embedding with the dose value and unit
        combined = np.concatenate((presc, np.array([dose_val, dose_unit])))
        prescriptions.append(combined)
    
    # Convert list to numpy array
    prescriptions = np.array(prescriptions)
    print(prescriptions.shape)

    return prescriptions

In [8]:
# function that adds the proper padding to our input arrays
def add_padding(prescriptions, pre_treatment, post_treatment):
    padded_prescriptions = np.pad(prescriptions, ((0, 0), (0, 50)), mode='constant')
    padded_pre_treatment = np.pad(pre_treatment, ((0, 0), (126, 25)), mode='constant')
    padded_post_treatment = np.pad(post_treatment, ((0, 0), (151, 0)), mode='constant')
    
    return padded_prescriptions, padded_pre_treatment, padded_post_treatment

In [9]:
# function that does the good stuff
def train_model(df, model):
    # Get unique patient/date pairs
    patient_date_pairs = get_unique_pairs(df)
    # Iterate through the patient/date pairs
    for patient in patient_date_pairs:
        for date in patient_date_pairs[patient]:
            # Get the data for the current patient/date pair
            patient_data = df[(df['subject_id'] == patient) & (df['prescription_start'] == date)]

            # drop unnecessary columns
            patient_data = patient_data.drop(['subject_id', 'prescription_start', 'pre_charttime', 'post_charttime'], axis=1)
            
            # Get the prescription input for the current patient/date pair (2DArray with shape (num_prescriptions, 126))
            prescriptions = get_presc_input(patient_data)

            # pre_treatment and post_treatment are 1D arrays
            pre_treatment = np.array(patient_data[[col for col in patient_data.columns if col.startswith('pre_')]].values[0])
            post_treatment = np.array(patient_data[[col for col in patient_data.columns if col.startswith('post_')]].values[0])

            # add padding to the inputs
            padded_prescriptions, padded_pre_treatment, padded_post_treatment = add_padding(prescriptions, pre_treatment, post_treatment)

            # Create the full sequence (1 patient, 3 time steps, 176 features)
            X_train = np.array([[
                padded_pre_treatment,           # Time Step 1: Pre-Treatment
                padded_prescriptions,           # Time Step 2: Prescription
                padded_post_treatment           # Time Step 3: Post-Treatment
            ]])

            y_train = X_train[:, -1, :]  # Target is the last time step (Post-Treatment)

            # Check data shape
            print("X_train shape:", X_train.shape)  # (1, 3, 176)
            print("y_train shape:", y_train.shape)  # (1, 176)

            # Train the model
            model.fit(X_train, y_train, epochs=50, batch_size=1)

            

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Define LSTM model
model = Sequential([
    LSTM(64, return_sequences=False, input_shape=(3, 176)),  # 3 time steps, 176 features per time step
    Dropout(0.2),
    LSTM(32, return_sequences=False),
    Dropout(0.2),
    Dense(16, activation="relu"),
    Dense(176)  # Predicting post-treatment blood profile (176 values but actually 25 values)
])

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Print model summary
model.summary()

train_model(df, model)




In [ ]:
# Test patient (pre-treatment + prescription, but post-treatment is missing)
X_test = np.array([[
    [12.5, 4500, 13.2, 4.5, 140],  # Pre-Treatment
    [0.0, 1.0, 500, 0.0, 0.0],     # Prescription (padded to 5 features)
    [0, 0, 0, 0, 0]  # Unknown post-treatment (to be predicted)
]])

# Predict post-treatment
predicted_post_treatment = model.predict(X_test)
print("Predicted Post-Treatment Blood Profile:", predicted_post_treatment)